# Sentiment analysis with TFLearn
Sentiment analysis using TFLearn / NLTK

## Dependencies

In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tflearn
from tflearn.data_utils import to_categorical

### Counting word frequency

In [3]:
from collections import Counter
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
import string

stemmer = nltk.PorterStemmer() #convert words to their word stem
punctuations = list(string.punctuation) #remove punctuations
stopwords_col = stopwords.words('english') #remove meaningless wrods

total_counts = Counter()#words counter
for text in reviews[0]:
        for word in [word for word in word_tokenize(text) if word not in stopwords_col and word not in punctuations]:
            try:
                raw = stemmer.stem(word)
            except:
                raw = word
        total_counts[raw] += 1
print("Total words in data set: ", len(total_counts))

Total words in data set:  4858


## Bag of words

In [5]:
vocab = sorted(total_counts, key=total_counts.get, reverse=True)
print(vocab[:1000])

['br', 'movi', 'film', 'time', 'one', 'watch', 'see', 'recommend', 'enjoy', 'rate', 'star', 'like', 'disappoint', 'end', 'good', 'better', 'give', 'life', 'miss', 'fan', 'bad', 'stori', 'entertain', 'well', 'seen', 'much', 'way', 'work', 'ever', 'instead', 'year', 'avoid', 'show', 'love', 'back', 'cost', 'day', 'made', 'money', 'classic', 'fun', 'dvd', 'view', 'best', 'look', 'think', 'deserv', 'check', 'warn', 'know', 'get', 'though', 'cinema', 'els', 'today', 'away', 'laugh', 'crap', 'thing', 'grade', 'funni', 'come', 'go', 'audienc', 'right', 'say', 'minut', 'world', 'make', 'comedi', 'perform', 'act', 'great', 'peopl', 'score', 'scene', 'bore', 'either', 'book', 'hour', 'thank', 'genr', 'pictur', 'flick', 'happen', 'anyon', 'point', 'interest', 'night', 'screen', 'seri', 'sequel', 'experi', 'ten', 'com', 'us', 'rent', 'kid', 'version', 'opinion', 'chanc', 'review', 'b', 'stuff', 'effort', 'futur', 'age', 'worth', 'aw', 'releas', 'gener', 'wrong', 'masterpiec', 'director', 'episod',

What's the last word in our vocabulary? We can use this to judge if 10000 is too few. If the last word is pretty common, we probably need to keep more words.

## Word to vectors

In [7]:
word2idx = {}
for index, word in enumerate(vocab):
    word2idx[word] = index

In [8]:
print(word2idx)

{'preach': 3830, 'travesti': 843, 'blom': 4365, 'intend': 481, 'mafia': 2109, 'nekromantik': 2110, 'king': 482, 'form': 348, 'necropoli': 2129, 'hat': 626, 'grate': 1369, 'menstruat': 2112, 'lack': 4256, 'pronto': 2114, 'mash': 2115, 'kit': 4549, 'poorli': 1613, 'carrier': 2116, 'thanx': 2893, 'warrior': 2117, 'reveal': 2118, 'fleapit': 2119, 'cameron': 1370, 'realist': 721, 'impos': 2120, 'especi': 4827, 'adam': 1039, 'heat': 4390, 'sourc': 1040, 'breaker': 4715, 'found': 1041, 'philip': 2123, 'appet': 3964, 'crisi': 2217, 'jack': 2124, 'rage': 2125, 'corpor': 1974, 'anim': 270, 'wilk': 3452, 'like': 11, 'boil': 1372, 'rant': 2130, 'desper': 1042, 'northwest': 2131, 'appar': 844, 'left': 722, 'finlay': 2132, 'stand': 845, 'assist': 1406, 'catchi': 3926, 'fantast': 446, 'horrorfest': 1373, 'jf': 2800, 'hit': 409, 'essenti': 1374, 'ontkean': 2137, 'testicl': 2138, 'glow': 2139, 'cleavag': 1375, 'aunt': 2140, 'era': 632, 'heartbeat': 1426, 'ether': 2143, 'barri': 2144, 'irk': 2145, 'four

## Text to vector function

In [9]:
def text_to_vector(text):
    to_vector = np.zeros((1, len(word2idx)), dtype=np.int)
    for word in [word for word in word_tokenize(text) if word not in stopwords_col and word not in punctuations]:
        try:
            raw = stemmer.stem(word)
        except:
            raw = word
        if raw in word2idx:
            to_vector[0][word2idx[raw]] += 1
    return to_vector

The result should be returned by this function :
```
text_to_vector('The tea is for a party to celebrate '
               'the movie so she has no time for a cake')[:65]
                   
array([0, 1, 0, 0, 2, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0])
```       

## Turn all of the sentances to Vectors

In [11]:
word_vectors = np.zeros((len(reviews), len(vocab)), dtype=np.int_)
for ii, (_, text) in enumerate(reviews.iterrows()):
    word_vectors[ii] = text_to_vector(text[0])

In [12]:
# Printing out the first 5 word vectors
word_vectors[:5, :23]

array([[0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0],
       [8, 0, 2, 0, 1, 0, 2, 0, 0, 0, 1, 5, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1],
       [8, 0, 6, 4, 4, 0, 2, 0, 0, 0, 1, 4, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 2],
       [0, 1, 0, 2, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]])

## Train, Validation, Test sets

Split the data into train, validation, and test sets. Using the function `to_categorical` from TFLearn to reshape the target data so that we'll have two output units and can classify with a softmax activation function. 

In [13]:
Y = (labels=='positive').astype(np.int_)
records = len(labels)

shuffle = np.arange(records)
np.random.shuffle(shuffle)
test_fraction = 0.9

train_split, test_split = shuffle[:int(records*test_fraction)], shuffle[int(records*test_fraction):]
trainX, trainY = word_vectors[train_split,:], to_categorical(Y.values[train_split], 2)
testX, testY = word_vectors[test_split,:], to_categorical(Y.values[test_split], 2)

## The network

In [15]:
# Network building
def build_model():
    # This resets all parameters and variables, leave this here
    tf.reset_default_graph()
    
    #### Your code ####
    net = tflearn.input_data([None, 4858])                          # Input
    net = tflearn.fully_connected(net, 5, activation='ReLU')      # Hidden
    net = tflearn.fully_connected(net, 2, activation='softmax')   # Output
    net = tflearn.regression(net, optimizer='sgd', learning_rate=0.1, loss='categorical_crossentropy')
    model = tflearn.DNN(net)
    return model

## Intializing the model

In [16]:
model = build_model()

## Training the network

In [19]:
# Training
model.fit(trainX, trainY, validation_set=0.1, show_metric=True, batch_size=128, n_epoch=20)

Training Step: 4769  | total loss: 0.24228 | time: 1.896s
| SGD | epoch: 030 | loss: 0.24228 - acc: 0.9182 -- iter: 20224/20250
Training Step: 4770  | total loss: 0.22824 | time: 2.928s
| SGD | epoch: 030 | loss: 0.22824 - acc: 0.9233 | val_loss: 0.36523 - val_acc: 0.8760 -- iter: 20250/20250
--


## Testing
This shows the accuracy of our model.

In [20]:
predictions = (np.array(model.predict(testX))[:,0] >= 0.5).astype(np.int_)
test_accuracy = np.mean(predictions == testY[:,0], axis=0)
print("Test accuracy: ", test_accuracy)

Test accuracy:  0.8636


## Test it out!

In [40]:
# Helper function that uses your model to predict sentiment
def test_sentence(sentence):
    positive_prob = model.predict(text_to_vector(sentence))[0][1]
    print('Sentence: {}'.format(sentence))
    print('P(positive) = {:.3f} :'.format(positive_prob), 
          'Positive' if positive_prob > 0.5 else 'Negative')

In [41]:
sentence = "Moonlight is by far the best movie of 2016."
test_sentence(sentence)
text_to_vector(sentence)
sentence = "It's amazing anyone could be talented enough to make something this spectacularly awful"
test_sentence(sentence)

Sentence: Moonlight is by far the best movie of 2016.
P(positive) = 0.621 : Positive
Sentence: It's amazing anyone could be talented enough to make something this spectacularly awful
P(positive) = 0.105 : Negative


In [43]:
test_sentence("This is just a masterpiece")

Sentence: This is just a masterpiece
P(positive) = 0.716 : Positive
